In [ ]:
%pip install transformers sentencepiece datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset
import numpy as np
# import tensorflow as tf
import torch
# from torch import optim
# from torch.nn import functional as F
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq #, AdamW
import evaluate
# from tqdm.notebook import tqdm
# import sentencepiece as spm

In [ ]:
# Load pre-trained model and tokenizer
model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Test pre-trained model
input_ids = tokenizer('I will buy some <extra_id_0> for <extra_id_1>', return_tensors='pt').input_ids
outputs = model.generate(input_ids)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<extra_id_0> clothes <extra_id_1> you.


In [ ]:
# Prepare dataset
prefix = "translate Nepali to English: "
src_lang = "ne_NP"
tgt_lang = "en_XX"
with open("train.ne_NP", "r", encoding='utf-8') as f:
    train_ne = [l.strip() for l in f.readlines()]
with open("train.en_XX", "r", encoding='utf-8') as f:
    train_en = [l.strip() for l in f.readlines()]
with open("test.ne_NP", "r", encoding='utf-8') as f:
    test_ne = [l.strip() for l in f.readlines()]
with open("test.en_XX", "r", encoding='utf-8') as f:
    test_en = [l.strip() for l in f.readlines()]
train_dataset_dict = {
    "ne_NP": train_ne,
    "en_XX": train_en
}
test_dataset_dict = {
    "ne_NP": test_ne,
    "en_XX": test_en
}
train_dataset = Dataset.from_dict(train_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

def preprocess_function(examples):
    return tokenizer([prefix + example for example in examples[src_lang]], text_target=examples[tgt_lang], max_length=128, truncation=True)

tokenized_train_inputs = train_dataset.map(preprocess_function, batched=True, remove_columns=[src_lang, tgt_lang])
tokenized_test_inputs = test_dataset.map(preprocess_function, batched=True, remove_columns=[src_lang, tgt_lang])

In [ ]:
print(len(train_dataset))
print(tokenized_train_inputs[0])
print(tokenizer.decode(tokenized_train_inputs[0]["input_ids"][2]))
print(tokenized_train_inputs.features)
print(tokenizer.decode(tokenized_train_inputs[0]["input_ids"], skip_special_tokens=True))
print(tokenizer.decode(tokenized_train_inputs[0]["labels"], skip_special_tokens=True))

3000
{'input_ids': [37194, 36403, 266, 288, 5413, 267, 2745, 3863, 69879, 57028, 135620, 14625, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [76978, 262, 1361, 259, 84984, 1]}
i
{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
translate Nepali to English: पपुआ न्यू गिनि
Papua New Guinea


In [ ]:
# Set up collator and metrics
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)#, return_tensors="np")
# tf_train_dataset = model.prepare_tf_dataset(
    # train_dataset,
    # collate_fn=data_collator,
    # batch_size=8,
    # shuffle=True
# )
metric = evaluate.load("bleu")

def postprocess_text(preds, labels):
    # remove `.split()` if not using bleu
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["bleu"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mt5_ne_en",
    evaluation_strategy="epoch",
    # evaluation_strategy="steps",
    # eval_steps=1000,
    # learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    # fp16=True,
    # bf16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_inputs,
    eval_dataset=tokenized_test_inputs,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(torch.optim.Adam(model.parameters(), lr=0.001), None)
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,6.215300,6.090819,0.000000,13.990000
2,3.383400,5.594716,0.000000,16.900000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1500, training_loss=4.68063232421875, metrics={'train_runtime': 805.577, 'train_samples_per_second': 7.448, 'train_steps_per_second': 1.862, 'total_flos': 423450635452416.0, 'train_loss': 4.68063232421875, 'epoch': 2.0})

In [ ]:
# torch.cuda.empty_cache()
trainer.save_model("mt5_ne_en_final")

In [ ]:
finetuned_model = MT5ForConditionalGeneration.from_pretrained("mt5_ne_en_final")

In [ ]:
for i in range(0,10):
    finetune_input = tokenizer.decode(tokenized_train_inputs[i]["input_ids"], skip_special_tokens=True)
    print("Input: ", finetune_input)
    print("Expected: ", tokenizer.decode(tokenized_train_inputs[i]["labels"], skip_special_tokens=True))
    input_ids = tokenizer(finetune_input, return_tensors='pt').input_ids
    outputs = finetuned_model.generate(input_ids)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Prediction: ", output_text)


Input:  पपुआ न्यू गिनि
Expected:  Papua New Guinea
Prediction:  _New Folder
Input:  "% 1" छवि लोड गर्न असफल
Expected:  Failed to load image "%1"
Prediction:  _Always displayed
Input:  दुईवटा नम्बरहरूको गुणनफल प्रदर्शित भयो। बराबरी साइनको दायाँपट्टि, वस्तुको उत्तर दिनुहोस्। तपाईँको उत्तरलाई परिमार्जन गर्न बायाँ र दायाँ बाँणहरू प्रयोग गर्नुहोस् र तपाईँले गरेको सही भयो भएन जाँच्न 'प्रविष्ट गर्नुहोस्' कुञ्जी थिच्नुहोस्। यदि सही छैन भने, पुन प्रयास गर्नुहोस्।
Expected:  A multiplication of two numbers is displayed. At the right of the equals sign, give the answer, the product. Use the left and right arrows to modify your answer and press the Enter key to check if you've got it right. If not, just try again.
Prediction:  Enable a new item.
Input:  ग्रीड रङ:
Expected:  Grid color:
Prediction:  _New Folder
Input:  दुरि:
Expected:  Distance:
Prediction:  _New Folder
Input:  यो घटना वा गर्नुपर्ने कार्य दोहोरिनु पर्ने महिनाको दिन ।
Expected:  The day of the month on which this event or to-do shou

In [ ]:
def translate(nepali):
    input_ids = tokenizer(prefix+nepali, return_tensors='pt').input_ids
    outputs = finetuned_model.generate(input_ids)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text


In [ ]:
print(translate("फाइलमा प्रयोगकर्ताहरूको सूची बचत गर्नुहोस्"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


_Always displayed


In [ ]:
# Test fine-tuned model
input_ids = tokenizer('translate Nepali to English: प्याच गरिने फाइल/डाइरेक्टरी', return_tensors='pt').input_ids
outputs = finetuned_model.generate(input_ids)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


_Filename:


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# output_dir = '/content/drive/MyDrive/translation_model'
# model.save_pretrained(output_dir)

Mounted at /content/drive
